# LDA2Vec
Adapted from tensorflow implementation: 

(Current) https://github.com/nateraw/Lda2vec-Tensorflow

(Old) https://github.com/meereeum/lda2vec-tf

(Original) https://github.com/cemoody/lda2vec

In [1]:
import pandas as pd
import numpy as np
import pyLDAvis
import pickle

In [2]:
import sys
sys.path.append('./Lda2vec-Tensorflow-master')

In [3]:
from lda2vec import *

Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
import tensorflow as tf
from tensorflow.keras import layers
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0Metal device set to: Apple M1



2021-11-20 00:26:43.994139: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-11-20 00:26:43.994227: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Find Ending Index of Documents for Each Year

In [5]:
data = pd.read_csv("papers.csv")

In [6]:
data = data.sort_values(by=["year"], kind='mergesort')

In [7]:
years = list(data["year"].unique())
counts = list(data["year"].value_counts())[::-1]
for i in range(1,len(counts)):
  counts[i] += counts[i-1]
year_index = {years[i]:counts[i] for i in range(len(years))}

In [8]:
year_index

{1987: 90,
 1988: 184,
 1989: 285,
 1990: 412,
 1991: 552,
 1992: 695,
 1993: 839,
 1994: 989,
 1995: 1139,
 1996: 1290,
 1997: 1442,
 1998: 1594,
 1999: 1746,
 2000: 1904,
 2001: 2101,
 2002: 2299,
 2003: 2503,
 2004: 2710,
 2005: 2917,
 2006: 3124,
 2007: 3341,
 2008: 3591,
 2009: 3853,
 2010: 4145,
 2011: 4451,
 2012: 4811,
 2013: 5179,
 2014: 5582,
 2015: 5993,
 2016: 6562,
 2017: 7241}

## Preprocessing

$\tt cleaned.txt$ is a txt file with all NIPS papers with all stop words and words with $\rm{length} \leq 3$ removed.

Preprocess incorporates tokenization (splitting sentences into words), creating a vocabulary to save mappings from tokens to integer indices, and generating skip-grams.

In [9]:
df = pd.read_csv("cleaned.txt")
df = df[df["stop_removed_paper_text"].notnull()]

In [10]:
df = df[4451:4551] # Use only 2011 papers

In [11]:
# Initialize a preprocessor
P = nlppipe.Preprocessor(df, "stop_removed_paper_text", max_features=30000, maxlen=10000, min_count=30)

/Users/keanl/miniforge3/lib/python3.9/site-packages/spacy/util.py:833: UserWarning: [W095] Model 'en_core_web_sm' (3.1.0) was trained with spaCy v3.1 and may not be 100% compatible with the current version (3.2.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [12]:
# Run the preprocessing on your dataframe
P.preprocess()


---------- Tokenizing Texts ----------


100it [00:35,  2.80it/s]


Removing 12703 low frequency tokens out of 14216 total tokens

---------- Getting Skipgrams ----------


100it [00:02, 49.93it/s]


In [13]:
# Should we load pretrained embeddings from file
load_embeds = True

# Load embeddings from file if we choose to do so
if load_embeds:
    # Load embedding matrix from file path - change path to where you saved them
    embedding_matrix = P.load_glove("glove.6B.300d.txt")
else:
    embedding_matrix = None

/Users/keanl/miniforge3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [14]:
# Save data to data_dir
P.save_data("clean_data11", embedding_matrix=embedding_matrix)

## Using the LDA2Vec Model

Using the LDA2Vec model on preprocessed data.

In [5]:
# Path to preprocessed data
data_path  = "clean_data"
# Whether or not to load saved embeddings file
load_embeds = True

# Load data from files
(idx_to_word, word_to_idx, freqs, pivot_ids,
 target_ids, doc_ids, embed_matrix) = utils.load_preprocessed_data(data_path, load_embed_matrix=load_embeds)

In [6]:
# Number of unique documents
num_docs = doc_ids.max() + 1
# Number of unique words in vocabulary (int)
vocab_size = len(freqs)
# Embed layer dimension size
# If not loading embeds, change 128 to whatever size you want.
embed_size = embed_matrix.shape[1] if load_embeds else 128
# Number of topics to cluster into
num_topics = 20
# Amount of iterations over entire dataset
num_epochs = 50
# Batch size - Increase/decrease depending on memory usage
batch_size = 4096
# Epoch that we want to "switch off" regularization
switch_loss_epoch = 0
# Pretrained embeddings value
pretrained_embeddings = embed_matrix if load_embeds else None
# If True, save logdir, otherwise don't
save_graph = False

In [7]:
# Initialize the model
m = model(num_docs,
          vocab_size,
          num_topics,
          embedding_size=embed_size,
          pretrained_embeddings=pretrained_embeddings,
          freqs=freqs,
          batch_size = batch_size,
          save_graph_def=save_graph)

2021-11-20 00:26:51.045716: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-11-20 00:26:51.045773: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2021-11-20 00:26:51.204577: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-11-20 00:26:51.204703: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [8]:
# Train the model
m.train(pivot_ids,
        target_ids,
        doc_ids,
        len(pivot_ids),
        num_epochs,
        idx_to_word=idx_to_word,
        switch_loss_epoch=switch_loss_epoch)

2021-11-20 00:26:51.871949: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-11-20 00:26:51.894388: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-11-20 00:26:51.900358: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.



EPOCH: 1


2021-11-20 00:27:17.027727: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


LOSS 1577.5953 w2v 14.610157 lda 1562.9852 reg 64.68488

EPOCH: 2


2021-11-20 00:27:41.709085: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


...

2021-11-20 00:36:32.079418: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


LOSS 472.03183 w2v 5.6121206 lda 466.4197 reg 77.34589

EPOCH: 25


2021-11-20 00:36:56.442789: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-11-20 00:36:56.461123: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


LOSS 470.79916 w2v 4.3794374 lda 466.41974 reg 77.34739
---------Closest 10 words to given indexes----------
Topic 0 : orig, koray, sherjil, jaitly, entailment, mirza, kavukcuoglu, adjustable, xiangyu, diederik
Topic 1 : sherjil, lanctot, abadie, koray, rusu, veness, volodymyr, antonoglou, hassabis, guez
Topic 2 : saliency, houdini, imagenet, ozair, localisation, xiangyu, entailment, attacks, jaitly, abadie
Topic 3 : abadie, bound, note, sherjil, given, consider, citro, mirza, entailment, ozair
Topic 4 : sdca, sadmm, admm, entailment, jimmy, linearized, abadie, kingma, shakir, diederik
Topic 5 : jaitly, sherjil, warde, curran, ozair, bilmes, factorized, abadie, florina, icml
Topic 6 : ozair, farley, sherjil, warde, abadie, mirza, dumoulin, mehdi, osindero, veness
Topic 7 : ozair, sherjil, entailment, warde, rusu, farley, antonoglou, abadie, mastering, koren
Topic 8 : ranganath, sherjil, chivi, abadie, nonconvex, lafferty, warde, ozair, tong, danilo
Topic 9 : entailment, warde, ozair, j

KeyboardInterrupt: 

## Get Word and Topic Embeddings

Visualize topics

In [9]:
idx_to_word

{1: 'learning',
 2: 'model',
 3: 'data',
 4: 'algorithm',
 5: 'function',
 6: 'neural',
 7: 'time',
 8: 'training',
 9: 'problem',
 10: 'information',
 11: 'results',
 12: 'number',
 13: 'distribution',
 14: 'models',
 15: 'networks',
 16: 'network',
 17: 'based',
 18: 'figure',
 19: 'methods',
 20: 'matrix',
 21: 'method',
 22: 'given',
 23: 'algorithms',
 24: 'pages',
 25: 'different',
 26: 'gradient',
 27: 'arxiv',
 28: 'performance',
 29: 'loss',
 30: 'optimization',
 31: 'linear',
 32: 'work',
 33: 'section',
 34: 'image',
 35: 'error',
 36: 'approach',
 37: 'case',
 38: 'theorem',
 39: 'deep',
 40: 'state',
 41: 'random',
 42: 'bound',
 43: 'conference',
 44: 'machine',
 45: 'optimal',
 46: 'systems',
 47: 'probability',
 48: 'parameters',
 49: 'functions',
 50: 'large',
 51: 'following',
 52: 'input',
 53: 'space',
 54: 'samples',
 55: 'analysis',
 56: 'processing',
 57: 'size',
 58: 'value',
 59: 'sample',
 60: 'convex',
 61: 'stochastic',
 62: 'proposed',
 63: 'dataset',
 64: 

In [10]:
doc_embed = m.sesh.run(m.mixture.doc_embedding)

2021-11-20 00:37:19.456109: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [11]:
topic_embed = m.sesh.run(m.mixture.topic_embedding)

2021-11-20 00:37:20.075586: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [12]:
word_embed = m.sesh.run(m.w_embed.embedding)

2021-11-20 00:37:20.518781: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [13]:
# Extract all unique words in order of index 0-vocab_size
vocabulary = []
for k,v in idx_to_word.items():
    vocabulary.append(v)

In [14]:
doc_lengths = np.load("clean_data11" + "/doc_lengths.npy")

In [15]:
vis_data = utils.prepare_topics(doc_embed, topic_embed, word_embed, np.array(vocabulary), doc_lengths=doc_lengths,
                              term_frequency=freqs, normalize=True)

In [16]:
prepared_vis_data = pyLDAvis.prepare(**vis_data)

ValidationError: 
 * Length of doc_lengths not equal to the number of rows in doc_topic_dists;both should be equal to the number of documents in the data.

In [17]:
pyLDAvis.display(prepared_vis_data)

NameError: name 'prepared_vis_data' is not defined

In [27]:
np.save("doc_embed11", doc_embed)

In [28]:
np.save("word_embed11", word_embed)

In [29]:
np.save("topic_embed11", topic_embed)